# 模型

## 微调地区+模型+Morris

### 修改

In [1]:
import pandas as pd, numpy as np, os, re, psutil, shap, matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from interpret.blackbox import MorrisSensitivity
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    r2_score, explained_variance_score,
    mean_squared_error, mean_absolute_error, median_absolute_error
)
import matplotlib

# —— 字体配置：宋体 + Times ——
config = {
    "font.family": 'serif',
    "font.serif": ['SimSun'],
    "mathtext.fontset": 'stix',
    "font.size": 12,
    "axes.unicode_minus": False
}
matplotlib.rcParams.update(config)

# —— 路径配置 ——
INPUT = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\data\var_attri_data_interp_cleaned.csv"
OUTPUT_DIR = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def sanitize(s):
    return re.sub(r'[<>:"/\\|?*]', '_', s)

def check_memory(gb=1):
    avail = psutil.virtual_memory().available / (1024**3)
    if avail < gb:
        raise MemoryError(f"内存不足: {avail:.2f} GB")
        
def analyze(var, df, output_dir, fine_tune_regions=["CHN","R10CHINA+"], target_model=None):
    id_cols = ['Model','Scenario','Region','Year', var]
    vars_cols = [
        c for c in df.columns
        if c not in id_cols and var not in c
    ]
    data = df[id_cols + vars_cols].dropna()

    # —— 模型子样本筛选 ——
    if target_model is not None:
        data = data[data['Model'] == target_model]
        print(f"[√] 仅针对模型 {target_model} 进行子样本分析，样本量: {len(data)}")
        if data.empty:
            print(f"[!] 未找到模型 {target_model} 的数据，跳过。")
            return

    X, y = data[vars_cols], data[var].values

    # —— 仅针对 fine_tune_regions 筛选
    mask = data['Region'].isin(fine_tune_regions)
    X_sub, y_sub = X[mask], y[mask]

    # —— 模型: RandomForest ——
    et = RandomForestRegressor(
        n_estimators=100,
        max_depth=8,
        min_samples_leaf=4,
        max_features="sqrt",
        random_state=42,
        n_jobs=-1
    )
    et.fit(X_sub, y_sub)

    check_memory()

    # ——— 性能评估
    Xtr, Xte, ytr, yte = train_test_split(X_sub, y_sub, test_size=0.3, random_state=42)
    preds_te = et.predict(Xte)
    metrics = {
        "R2_in": r2_score(ytr, et.predict(Xtr)),
        "R2_out": r2_score(yte, preds_te),
        "EVS_out": explained_variance_score(yte, preds_te),
        "MSE_out": mean_squared_error(yte, preds_te),
        "MAE_out": mean_absolute_error(yte, preds_te),
        "MedAE_out": median_absolute_error(yte, preds_te),
    }
    tag = sanitize(var)
    outdir = os.path.join(output_dir, tag + (f"_model_{sanitize(target_model)}" if target_model else ""))
    os.makedirs(outdir, exist_ok=True)
    pd.DataFrame([metrics]).to_csv(
        os.path.join(outdir, f"{tag}_模型指标.csv"),
        index=False, encoding='utf-8-sig'
    )

    # ——— 特征重要性
    imp = et.feature_importances_
    imp_df = pd.DataFrame({
        "变量": vars_cols,
        "相对重要性(%)": np.round(imp / imp.sum() * 100, 4)
    }).nlargest(10, "相对重要性(%)")
    imp_df.to_csv(
        os.path.join(outdir, f"{tag}_变量相对重要性.csv"),
        index=False, encoding='utf-8-sig'
    )

    # ——— SHAP 分析
    sample = X_sub.sample(n=min(100, len(X_sub)), random_state=42)
    expl = shap.TreeExplainer(et)
    shap_vals = expl.shap_values(sample)

    top_feats = imp_df["变量"].tolist()
    top_idx = [sample.columns.get_loc(f) for f in top_feats]

    shap.summary_plot(
        shap_vals[:, top_idx],
        sample[top_feats],
        feature_names=top_feats,
        show=False, max_display=10
    )
    fig = plt.gcf()
    ax = fig.axes[0]; ax.set_xlabel(""); ax.set_ylabel("")
    cbar = fig.axes[-1]; cbar.set_ylabel(""); cbar.set_yticklabels([]); cbar.set_xticklabels([])

    mean_shap = np.abs(shap_vals[:, top_idx]).mean(axis=0)
    ordered_idx = np.argsort(mean_shap)[::-1]
    for y_loc, idx in enumerate(ordered_idx):
        mv = mean_shap[idx]
        ax.hlines(y=y_loc, xmin=mv - 0.01, xmax=mv + 0.01,
                  color="black", linewidth=12, zorder=20)

    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{tag}_SHAP.png"), dpi=300)
    plt.close()
    pd.DataFrame({
        "变量": [top_feats[i] for i in ordered_idx],
        "SHAP均值": np.round(mean_shap[ordered_idx], 6)
    }).to_csv(
        os.path.join(outdir, f"{tag}_mean_SHAP.csv"),
        index=False, encoding='utf-8-sig'
    )

    mean_shap_df = pd.DataFrame({
        "变量": [top_feats[i] for i in ordered_idx],
        "均值_SHAP": np.round(mean_shap[ordered_idx], 6),
        "特征重要性": np.round(imp_df["相对重要性(%)"].values[ordered_idx] / 100, 6),
        "SHAP_标准差": np.round(np.std(shap_vals[:, top_idx][:, ordered_idx], axis=0), 6)
    })
    mean_shap_df.to_csv(os.path.join(outdir, f"{tag}_SHAP_vs_Imp.csv"),
                        index=False, encoding='utf-8-sig')

    # 📊 图1. SHAP vs 特征重要性柱状图
    fig, ax = plt.subplots(figsize=(8, 6))
    df_plot = mean_shap_df.sort_values("均值_SHAP", ascending=True)
    y = np.arange(len(df_plot))
    ax.barh(y - 0.2, df_plot["均值_SHAP"], height=0.4, label="SHAP均值")
    ax.barh(y + 0.2, df_plot["特征重要性"], height=0.4, label="特征重要性")
    ax.set_yticks(y)
    ax.set_yticklabels(df_plot["变量"])
    ax.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{tag}_SHAP_vs_Imp_bar.png"), dpi=300)
    plt.close()

    # 📈 图2. Morris µ* vs σ 散点图
    fig, ax = plt.subplots(figsize=(6, 6))
    sc = ax.scatter(
        mean_shap_df["均值_SHAP"],
        mean_shap_df["SHAP_标准差"],
        s=100 * mean_shap_df["特征重要性"],
        c=mean_shap_df["特征重要性"],
        cmap="viridis",
        alpha=0.8,
        edgecolors='k'
    )
    for _, row in mean_shap_df.iterrows():
        ax.text(row["均值_SHAP"], row["SHAP_标准差"], row["变量"], fontsize=8)
    ax.set_xlabel("Morris µ* (SHAP均值)")
    ax.set_ylabel("σ (SHAP标准差)")
    plt.colorbar(sc, label="特征重要性")
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{tag}_Morris_scatter.png"), dpi=300)
    plt.close()

    # 📉 图3. SHAP Dependence Plot（前 3 个变量）
    for i, feat in enumerate([top_feats[i] for i in ordered_idx[:3]]):
        interaction_index = 'auto'
        if interaction_index == 'auto':
            interactions = shap.utils.approximate_interactions(
                feat, shap_vals, sample
            )
            interaction_feat = sample.columns[interactions[0]]
        else:
            interaction_feat = interaction_index

        x = sample[feat].values
        y_ = shap_vals[:, sample.columns.get_loc(feat)]
        color = sample[interaction_feat].values if interaction_feat != feat else None

        shap.dependence_plot(
            sample.columns.get_loc(feat), shap_vals, sample,
            feature_names=sample.columns.tolist(),
            interaction_index=interaction_index, show=False
        )
        plt.tight_layout()
        png_path = os.path.join(outdir, f"{tag}_SHAP_dependence_{sanitize(feat)}.png")
        plt.savefig(png_path, dpi=300)
        plt.close()
        
        data = {
            feat: x,
            f"SHAP_{feat}": y_,
        }
        if color is not None:
            data[interaction_feat] = color
        df_dep = pd.DataFrame(data)
        csv_path = os.path.join(outdir, f"{tag}_SHAP_dependence_{sanitize(feat)}.csv")
        df_dep.to_csv(csv_path, index=False)

    print(f"[✔] {var} 使用 RandomForest 分析完成，结果保存在：{outdir}")

# ========== 用法示例 ==========
# df = pd.read_csv(INPUT)
# analyze('YourVariable', df, OUTPUT_DIR, target_model='GCAM5.3')

### Sobol 漏斗式验证

In [2]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import pandas as pd, numpy as np, os

def run_sobol_analysis(X, y, var_names, output_dir, tag="fei", target_model=None):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X[var_names])

    problem = {
        'num_vars': len(var_names),
        'names': var_names,
        'bounds': [[0, 1]] * len(var_names)
    }

    # 采样 + 模拟
    param_values = saltelli.sample(problem, 512, calc_second_order=True)
    # === 替换为 RandomForestRegressor ===
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=8,
        min_samples_leaf=4,
        max_features="sqrt",
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_scaled, y)
    # 注意：随机森林不支持直接超范围外推, 这里只是演示；实际sobol采样外推可能有误差
    Y = model.predict(param_values)

    # Sobol 分析
    sobol_result = sobol.analyze(problem, Y, calc_second_order=True, print_to_console=False)

    # 输出文件名支持模型名区分
    tag_full = f"{tag}_model_{sanitize(str(target_model))}" if target_model else tag
    df_out = pd.DataFrame({
        "变量": var_names,
        "一阶S1": np.round(sobol_result['S1'], 4),
        "总效ST": np.round(sobol_result['ST'], 4),
        "S1_conf": np.round(sobol_result['S1_conf'], 4),
        "ST_conf": np.round(sobol_result['ST_conf'], 4)
    })
    outpath = os.path.join(output_dir, f"{tag_full}_sobol.csv")
    df_out.to_csv(outpath, index=False, encoding='utf-8-sig')
    print(f"[✔] Sobol 总效应分析完成，结果已保存至：{outpath}")

def subgroup_shap_analysis_model(var, df, output_dir, fine_tune_regions):
    df_china = df[df['Region'].isin(fine_tune_regions)].copy()
    model_vals = df_china["Model"].unique()

    for model_name in model_vals:
        df_sub = df_china[df_china["Model"] == model_name].copy()
        if len(df_sub) < 40:
            print(f"[跳过] Model={model_name} 样本过少（{len(df_sub)} 条）")
            continue

        try:
            print(f"[▶] 正在分析 Model={model_name}...")
            sub_output = os.path.join(output_dir, f"China_Model_{sanitize(str(model_name))}")
            # analyze 函数需支持 target_model 参数
            analyze(var, df, sub_output, fine_tune_regions, target_model=model_name)
        except Exception as e:
            print(f"[❌] Model={model_name} 分析失败：{e}")

# 必须定义 sanitize 函数用于命名
def sanitize(s):
    import re
    return re.sub(r'[<>:"/\\|?*]', '_', s)

# analyze 函数即为之前你修改的支持 RandomForest/target_model 的 analyze

# 示例用法
# run_sobol_analysis(X, y, var_names, output_dir, tag="your_tag", target_model="GCAM5.3")
# subgroup_shap_analysis_model('变量名', df, output_dir, ["CHN", "R10CHINA+"])


# 变量三

## COFFEE 1.1	Primary Energy

In [3]:
df = pd.read_csv(INPUT)
analyze('pe', df, OUTPUT_DIR, target_model='COFFEE 1.1')

[√] 仅针对模型 COFFEE 1.1 进行子样本分析，样本量: 6930
[✔] pe 使用 RandomForest 分析完成，结果保存在：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1


### 子集特征再训练与验证

In [4]:
def evaluate_subset(var, df, feature_subset, subset_name, output_dir, model_name=None):
    id_cols = ['Model','Scenario','Region','Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    rf = RandomForestRegressor(
        n_estimators=40, max_depth=6,
        min_samples_leaf=5, max_features="sqrt",
        random_state=42, warm_start=True
    )
    rf.fit(X_all, y_all)

    # Step 1️⃣ 微调到中国区域
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        rf.set_params(n_estimators=60)
        rf.fit(X_china, y_china)

    # Step 2️⃣ 可选模型子样本微调
    if model_name:
        mask_model = (df['Model'] == model_name)
        if mask_model.sum() > 20:
            X_model, y_model = X_all[mask_model], y_all[mask_model]
            rf.set_params(n_estimators=80)
            rf.fit(X_model, y_model)

    # 评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, rf.predict(Xtr)),
        "R2_out": r2_score(yte, rf.predict(Xte)),
        "EVS_out": explained_variance_score(yte, rf.predict(Xte)),
        "MSE_out": mean_squared_error(yte, rf.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, rf.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, rf.predict(Xte)),
    }
    return metrics

In [5]:
linear_feats = ["emcoen", "eceaip"]
nonlinear_feats = ["fee", "see"]
shap_top5_feats = ["fee", "see", "emcoen", "eces", "seen","eceaip","seeg","fei","ecese","seec"]

In [6]:
target_var = 'pe'
model_name = "COFFEE 1.1"  # 👈 你传入的模型名
outdir = os.path.join(OUTPUT_DIR, "pe")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, linear_feats, "线性主效变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, nonlinear_feats, "非线性交互变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, shap_top5_feats, "SHAP前十变量", outdir, model_name))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, f"{model_name}_{target_var}_子集建模比较结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 子集建模评估完成，结果已保存至：", outdir)

✅ 子集建模评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe


### Sobol 漏斗式验证

In [11]:
import pandas as pd
df = pd.read_csv(r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\data\var_attri_data_interp_cleaned.csv")
# 假设你的变量名叫 'CO2'
X = df[["fee", "see"]]        # 用你关心的变量
y = df['pe']

output_dir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results"
var_names = ["fee", "see"]

# 针对模型子样本
df_gcam = df[df['Model'] == 'COFFEE 1.1']
X_gcam = df_gcam[var_names]
y_gcam = df_gcam['pe']
run_sobol_analysis(X_gcam, y_gcam, var_names, output_dir, tag="pe", target_model='COFFEE 1.1')


C:\Users\phc\AppData\Local\Temp\ipykernel_21308\2532936671.py:18: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 512, calc_second_order=True)


[✔] Sobol 总效应分析完成，结果已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe_model_COFFEE 1.1_sobol.csv


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


### 模型降维&交互建模

#### see

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def calc_metrics(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}EVS": explained_variance_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ['see']
target_var = 'pe'
model_name = "COFFEE 1.1"
outdir = os.path.join(OUTPUT_DIR, "pe")
os.makedirs(outdir, exist_ok=True)

X_all = df[feature_subset].dropna()
y_all = df.loc[X_all.index, target_var]

# === 划分训练集/测试集（按0.3比例） ===
from sklearn.model_selection import train_test_split
Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)

# === 1. 全训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 增量微调：中国子样本，再加50棵树 ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. 增量微调：模型子样本，再加50棵树 ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练集/测试集评估 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr), prefix="R2_in_")
metrics_out = calc_metrics(yte, rf.predict(Xte), prefix="R2_out_")

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2_in_R2"],
    "R2_out": metrics_out["R2_out_R2"],
    "EVS_out": metrics_out["R2_out_EVS"],
    "MSE_out": metrics_out["R2_out_MSE"],
    "MAE_out": metrics_out["R2_out_MAE"],
    "MedAE_out": metrics_out["R2_out_MedAE"],
}

summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF增量微调模型已在全部样本回归评估，结果已保存至：", outdir)

✅ RF增量微调模型已在全部样本回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe


#### see分段+see

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def calc_metrics(y_true, y_pred, prefix=""):
    return {
        f"{prefix}R2": r2_score(y_true, y_pred),
        f"{prefix}EVS": explained_variance_score(y_true, y_pred),
        f"{prefix}MSE": mean_squared_error(y_true, y_pred),
        f"{prefix}MAE": mean_absolute_error(y_true, y_pred),
        f"{prefix}MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === see 分三段哑变量 ===
see_thresh1, see_thresh2 = 25, 35
df['see_low']  = (df['see'] <=  see_thresh1).astype(int)
df['see_mid']  = ((df['see'] > see_thresh1) & (df['see'] <= see_thresh2)).astype(int)
df['see_high'] = (df['see'] >  see_thresh2).astype(int)
assert ((df[['see_low','see_mid','see_high']].sum(axis=1) == 1)).all()

feature_subset = ['see_low', 'see_mid', 'see_high', 'see']
target_var = 'pe'
model_name = "COFFEE 1.1"
outdir = os.path.join(OUTPUT_DIR, "pe")
os.makedirs(outdir, exist_ok=True)

X_all = df[feature_subset].dropna()
y_all = df.loc[X_all.index, target_var]

# === 划分训练集/测试集 ===
from sklearn.model_selection import train_test_split
Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)

# === 1. 全训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 增量微调：中国子样本，再加50棵树 ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. 增量微调：模型子样本，再加50棵树 ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 分别评估训练集和测试集 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr), prefix="R2_in_")
metrics_out = calc_metrics(yte, rf.predict(Xte), prefix="R2_out_")

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2_in_R2"],
    "R2_out": metrics_out["R2_out_R2"],
    "EVS_out": metrics_out["R2_out_EVS"],
    "MSE_out": metrics_out["R2_out_MSE"],
    "MAE_out": metrics_out["R2_out_MAE"],
    "MedAE_out": metrics_out["R2_out_MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF分段变量+see增量微调后模型已完成训练/测试集回归评估，结果已保存至：", outdir)

✅ RF分段变量+see增量微调后模型已完成训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe


#### see+fee

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ['see', 'fee']
target_var = 'pe'
model_name = "COFFEE 1.1"
outdir = os.path.join(OUTPUT_DIR, "pe")
os.makedirs(outdir, exist_ok=True)

# === 随机拆分训练集/测试集 ===
from sklearn.model_selection import train_test_split
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 全训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本微调，增量加树 ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. 模型子样本微调，再增量加树 ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练集/测试集分别评估 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2"],
    "R2_out": metrics_out["R2"],
    "EVS_out": metrics_out["EVS"],
    "MSE_out": metrics_out["MSE"],
    "MAE_out": metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF see+fee 增量微调后模型已完成训练/测试集回归评估，结果已保存至：", outdir)

✅ RF see+fee 增量微调后模型已完成训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe


#### see+fee+fee*see

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['fee_see'] = df['fee'] * df['see']

feature_subset = ['see', 'fee', 'fee_see']
target_var = 'pe'
model_name = "COFFEE 1.1"
outdir = os.path.join(OUTPUT_DIR, "pe")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练集/测试集 ===
from sklearn.model_selection import train_test_split
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 全训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本微调 ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. 模型子样本微调 ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 分别评估 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2"],
    "R2_out": metrics_out["R2"],
    "EVS_out": metrics_out["EVS"],
    "MSE_out": metrics_out["MSE"],
    "MAE_out": metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF see+fee+fee*see 增量微调后模型已完成训练/测试集回归评估，结果已保存至：", outdir)

✅ RF see+fee+fee*see 增量微调后模型已完成训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe


#### see+fee+see*seeh

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['see_seeh'] = df['see'] * df['seeh']

feature_subset = ['see', 'fee', 'see_seeh']
target_var = 'pe'
model_name = "COFFEE 1.1"
outdir = os.path.join(OUTPUT_DIR, "pe")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练集/测试集 ===
from sklearn.model_selection import train_test_split
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本微调（增量加树） ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. 模型子样本微调（再增量加树） ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练/测试集分别评估 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2"],
    "R2_out": metrics_out["R2"],
    "EVS_out": metrics_out["EVS"],
    "MSE_out": metrics_out["MSE"],
    "MAE_out": metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF see+fee+see*seeh 增量微调后模型已完成训练/测试集回归评估，结果已保存至：", outdir)

✅ RF see+fee+see*seeh 增量微调后模型已完成训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe


#### see+fee+fee*see+see*seeh

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['fee_see'] = df['fee'] * df['see']
df['see_seeh'] = df['see'] * df['seeh']

feature_subset = ['see', 'fee', 'fee_see', 'see_seeh']
target_var = 'pe'
model_name = "COFFEE 1.1"
outdir = os.path.join(OUTPUT_DIR, "pe")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练集/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本微调（增量加树） ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. 模型子样本微调（再增量加树） ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练/测试集分别评估 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2"],
    "R2_out": metrics_out["R2"],
    "EVS_out": metrics_out["EVS"],
    "MSE_out": metrics_out["MSE"],
    "MAE_out": metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF see+fee+fee*see+see*seeh 增量微调后模型已完成训练/测试集回归评估，结果已保存至：", outdir)

✅ RF see+fee+fee*see+see*seeh 增量微调后模型已完成训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\pe


# 变量二

## GCAM 5.3

### Final Energy|Industry

In [19]:
df = pd.read_csv(INPUT)
analyze('fei', df, OUTPUT_DIR, target_model='GCAM 5.3')

[√] 仅针对模型 GCAM 5.3 进行子样本分析，样本量: 4590
[✔] fei 使用 RandomForest 分析完成，结果保存在：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3


#### 子集特征再训练与验证

In [20]:
def evaluate_subset(var, df, feature_subset, subset_name, output_dir, model_name=None):
    id_cols = ['Model','Scenario','Region','Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    rf = RandomForestRegressor(
        n_estimators=40, max_depth=6,
        min_samples_leaf=5, max_features="sqrt",
        random_state=42, warm_start=True
    )
    rf.fit(X_all, y_all)

    # Step 1️⃣ 微调到中国区域
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        rf.set_params(n_estimators=60)
        rf.fit(X_china, y_china)

    # Step 2️⃣ 可选模型子样本微调
    if model_name:
        mask_model = (df['Model'] == model_name)
        if mask_model.sum() > 20:
            X_model, y_model = X_all[mask_model], y_all[mask_model]
            rf.set_params(n_estimators=80)
            rf.fit(X_model, y_model)

    # 评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, rf.predict(Xtr)),
        "R2_out": r2_score(yte, rf.predict(Xte)),
        "EVS_out": explained_variance_score(yte, rf.predict(Xte)),
        "MSE_out": mean_squared_error(yte, rf.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, rf.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, rf.predict(Xte)),
    }
    return metrics

In [21]:
linear_feats = ["seebwoc"]
nonlinear_feats = ["seec", "seegwoc"]
shap_top5_feats = ["seec", "seegwoc", "seecwoc", "seebwoc", "eces","eceaip","emcoen","ecese","eced","fel"]

In [22]:
target_var = 'fei'
model_name = "GCAM 5.3"  # 👈 你传入的模型名
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, linear_feats, "线性主效变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, nonlinear_feats, "非线性交互变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, shap_top5_feats, "SHAP前十变量", outdir, model_name))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, f"{model_name}_{target_var}_子集建模比较结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 子集建模评估完成，结果已保存至：", outdir)

✅ 子集建模评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei


#### Sobol 漏斗式验证

In [23]:
import pandas as pd
df = pd.read_csv(r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\data\var_attri_data_interp_cleaned.csv")
# 假设你的变量名叫 'CO2'
X = df[["seec", "seegwoc"]]        # 用你关心的变量
y = df['fei']

output_dir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results"
var_names = ["seec", "seegwoc"]

# 针对模型子样本
df_gcam = df[df['Model'] == 'GCAM 5.3']
X_gcam = df_gcam[var_names]
y_gcam = df_gcam['fei']
run_sobol_analysis(X_gcam, y_gcam, var_names, output_dir, tag="fei", target_model='GCAM 5.3')


C:\Users\phc\AppData\Local\Temp\ipykernel_21308\2532936671.py:18: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 512, calc_second_order=True)


[✔] Sobol 总效应分析完成，结果已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei_model_GCAM 5.3_sobol.csv


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


#### 模型降维&交互建模

##### seec

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ['seec']
target_var = 'fei'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练集/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本增量微调 ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. GCAM 5.3 子样本增量微调 ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练/测试集分别评估六项指标 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2"],
    "R2_out": metrics_out["R2"],
    "EVS_out": metrics_out["EVS"],
    "MSE_out": metrics_out["MSE"],
    "MAE_out": metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF seec→fei/GCAM5.3 增量微调后模型已完成训练/测试集回归评估，结果已保存至：", outdir)

✅ RF seec→fei/GCAM5.3 增量微调后模型已完成训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei


##### seec分段+seec

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 1. 确定分段阈值 ===
seec_thresh1, seec_thresh2 = 10, 15

# === 2. 构造三段哑变量 ===
df['seec_low']  = (df['seec'] <=  seec_thresh1).astype(int)
df['seec_mid']  = ((df['seec'] > seec_thresh1) & (df['seec'] <= seec_thresh2)).astype(int)
df['seec_high'] = (df['seec'] >  seec_thresh2).astype(int)

# === 3. 校验 ===
assert (df[['seec_low','seec_mid','seec_high']].sum(axis=1) == 1).all()

feature_subset = ['seec_low', 'seec_mid', 'seec_high', 'seec']
target_var = 'fei'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]

# === 训练/测试集拆分 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本增量微调 ===
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. GCAM 5.3 子样本增量微调 ===
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练/测试集分别评估六项指标 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2"],
    "R2_out": metrics_out["R2"],
    "EVS_out": metrics_out["EVS"],
    "MSE_out": metrics_out["MSE"],
    "MAE_out": metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF seec分段+seec 增量微调后模型已在训练/测试集完成回归评估，结果已保存至：", outdir)

✅ RF seec分段+seec 增量微调后模型已在训练/测试集完成回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei


##### seec+seegwoc

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ['seec', 'seegwoc']
target_var = 'fei'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]

# === 拆分训练/测试集 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 训练集上训练
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本微调
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. GCAM 5.3 子样本微调
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 输出训练集、测试集六项指标
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":  metrics_in["R2"],
    "R2_out": metrics_out["R2"],
    "EVS_out": metrics_out["EVS"],
    "MSE_out": metrics_out["MSE"],
    "MAE_out": metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF最终微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF seec+seegwoc 微调后模型已在训练/测试集完成回归评估，结果已保存至：", outdir)

✅ RF seec+seegwoc 微调后模型已在训练/测试集完成回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei


##### seec+seegwoc+seegwoc*fel

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['seegwoc_fel'] = df['seegwoc'] * df['fel']

feature_subset = ['seec', 'seegwoc', 'seegwoc_fel']
target_var = 'fei'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 全样本训练
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本微调（增量加树）
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. GCAM 5.3 子样本微调（再增量加树）
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 分别输出训练/测试集指标
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF最终微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF seec+seegwoc+seegwoc*fel 微调后模型已在训练/测试集完成回归评估，结果已保存至：", outdir)

✅ RF seec+seegwoc+seegwoc*fel 微调后模型已在训练/测试集完成回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei


##### seec+seegwoc+seec*eced

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['seec_eced'] = df['seec'] * df['eced']

feature_subset = ['seec', 'seegwoc', 'seec_eced']
target_var = 'fei'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 全样本训练
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本微调（增加50棵树）
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. GCAM 5.3 子样本微调（再增加50棵树）
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 训练/测试集分别回归评估
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF最终微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF seec+seegwoc+seec*eced 微调后模型已在训练/测试集回归评估，结果已保存至：", outdir)

✅ RF seec+seegwoc+seec*eced 微调后模型已在训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei


##### seec+seegwoc+seegwoc*fel+seec*eced

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互特征 ===
df['seegwocfel'] = df['seegwoc'] * df['fel']
df['seececed']   = df['seec'] * df['eced']

feature_subset = ['seec', 'seegwoc', 'seegwocfel', 'seececed']
target_var = 'fei'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fei")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 全样本训练
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本微调（增量加50棵树）
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. GCAM 5.3 子样本微调（再加50棵树）
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 分别评估训练/测试集
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF最终微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF seec+seegwoc+seegwocfel+seececed 微调后模型已在训练/测试集回归评估，结果已保存至：", outdir)

✅ RF seec+seegwoc+seegwocfel+seececed 微调后模型已在训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fei


### Final Energy|Transportation

In [15]:
df = pd.read_csv(INPUT)
analyze('fet', df, OUTPUT_DIR, target_model='GCAM 5.3')

[√] 仅针对模型 GCAM 5.3 进行子样本分析，样本量: 4590
[✔] fet 使用 RandomForest 分析完成，结果保存在：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3


#### 子集特征再训练与验证

In [17]:
def evaluate_subset(var, df, feature_subset, subset_name, output_dir, model_name=None):
    id_cols = ['Model','Scenario','Region','Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    rf = RandomForestRegressor(
        n_estimators=40, max_depth=6,
        min_samples_leaf=5, max_features="sqrt",
        random_state=42, warm_start=True
    )
    rf.fit(X_all, y_all)

    # Step 1️⃣ 微调到中国区域
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        rf.set_params(n_estimators=60)
        rf.fit(X_china, y_china)

    # Step 2️⃣ 可选模型子样本微调
    if model_name:
        mask_model = (df['Model'] == model_name)
        if mask_model.sum() > 20:
            X_model, y_model = X_all[mask_model], y_all[mask_model]
            rf.set_params(n_estimators=80)
            rf.fit(X_model, y_model)

    # 评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, rf.predict(Xtr)),
        "R2_out": r2_score(yte, rf.predict(Xte)),
        "EVS_out": explained_variance_score(yte, rf.predict(Xte)),
        "MSE_out": mean_squared_error(yte, rf.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, rf.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, rf.predict(Xte)),
    }
    return metrics

In [16]:
linear_feats = ['seeb','fel','penr','see']
nonlinear_feats = ['feg']
shap_top5_feats = ["seec", "seegwoc", "seecwoc", "seebwoc", "eces","eceaip","emcoen","ecese","eced","fel"]

In [18]:
target_var = 'fet'
model_name = "GCAM 5.3"  # 👈 你传入的模型名
outdir = os.path.join(OUTPUT_DIR, "fet")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, linear_feats, "线性主效变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, nonlinear_feats, "非线性交互变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, shap_top5_feats, "SHAP前十变量", outdir, model_name))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, f"{model_name}_{target_var}_子集建模比较结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 子集建模评估完成，结果已保存至：", outdir)

✅ 子集建模评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet


#### Sobol 漏斗式验证

In [19]:
import pandas as pd
df = pd.read_csv(r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\data\var_attri_data_interp_cleaned.csv")
# 假设你的变量名叫 'CO2'
X = df[['feg']]        # 用你关心的变量
y = df['fet']

output_dir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results"
var_names = ['feg']

# 针对模型子样本
df_gcam = df[df['Model'] == 'GCAM 5.3']
X_gcam = df_gcam[var_names]
y_gcam = df_gcam['fet']
run_sobol_analysis(X_gcam, y_gcam, var_names, output_dir, tag="fet", target_model='GCAM 5.3')


C:\Users\phc\AppData\Local\Temp\ipykernel_17212\2532936671.py:18: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 512, calc_second_order=True)


[✔] Sobol 总效应分析完成，结果已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet_model_GCAM 5.3_sobol.csv


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


#### 模型降维&交互建模

##### feg

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ['feg']
target_var = 'fet'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fet")
os.makedirs(outdir, exist_ok=True)

# === 拆分训练/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 全样本训练
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本增量微调
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. GCAM 5.3 子样本增量微调
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 分别评估训练/测试集
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF feg→fet/GCAM5.3 增量微调后模型已在训练/测试集回归评估，结果已保存至：", outdir)

✅ RF feg→fet/GCAM5.3 增量微调后模型已在训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet


##### feg三分段+feg

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造 feg 三分段哑变量 ===
df['feg_low']  = (df.feg <=  8).astype(int)
df['feg_med']  = ((df.feg > 8) & (df.feg <= 12)).astype(int)
df['feg_high'] = (df.feg > 12).astype(int)
assert (df[['feg_low', 'feg_med', 'feg_high']].sum(axis=1) == 1).all()

feature_subset = ['feg_low', 'feg_med', 'feg_high', 'feg']
target_var = 'fet'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fet")
os.makedirs(outdir, exist_ok=True)

# === 分训练/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 全样本训练
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本增量微调
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. GCAM 5.3 子样本增量微调
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 输出训练/测试集指标
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))
summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调全样本回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF feg分段+feg/GCAM5.3 增量微调后模型已在训练/测试集回归评估，结果已保存至：", outdir)

✅ RF feg分段+feg/GCAM5.3 增量微调后模型已在训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet


##### feg+feg*fel

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造交互项 ===
df['feg_fel'] = df['feg'] * df['fel']

feature_subset = ['feg', 'feg_fel']
target_var = 'fet'
model_name = "GCAM 5.3"
outdir = os.path.join(OUTPUT_DIR, "fet")
os.makedirs(outdir, exist_ok=True)

# === 分训练/测试集 ===
X = df[feature_subset].dropna()
y = df.loc[X.index, target_var]
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 全样本训练（用训练集）
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本增量微调
china_mask = df.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. GCAM 5.3 子样本增量微调
model_mask = (df.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 训练/测试集评估
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))
summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF feg+feg*fel/GCAM5.3 增量微调已分训练/测试集回归评估，结果已保存至：", outdir)

✅ RF feg+feg*fel/GCAM5.3 增量微调已分训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\fet


## IMACLIM 1.1

### Secondary Energy|Electricity

In [23]:
df = pd.read_csv(INPUT)
analyze('see', df, OUTPUT_DIR, target_model='IMACLIM 1.1')

[√] 仅针对模型 IMACLIM 1.1 进行子样本分析，样本量: 6120
[✔] see 使用 RandomForest 分析完成，结果保存在：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see_model_IMACLIM 1.1


#### 子集特征再训练与验证

In [24]:
def evaluate_subset(var, df, feature_subset, subset_name, output_dir, model_name=None):
    id_cols = ['Model','Scenario','Region','Year', var]
    df = df[id_cols + feature_subset].dropna()
    X_all, y_all = df[feature_subset], df[var].values

    rf = RandomForestRegressor(
        n_estimators=40, max_depth=6,
        min_samples_leaf=5, max_features="sqrt",
        random_state=42, warm_start=True
    )
    rf.fit(X_all, y_all)

    # Step 1️⃣ 微调到中国区域
    mask_china = df['Region'].isin(["CHN", "R10CHINA+"])
    if mask_china.sum() > 30:
        X_china, y_china = X_all[mask_china], y_all[mask_china]
        rf.set_params(n_estimators=60)
        rf.fit(X_china, y_china)

    # Step 2️⃣ 可选模型子样本微调
    if model_name:
        mask_model = (df['Model'] == model_name)
        if mask_model.sum() > 20:
            X_model, y_model = X_all[mask_model], y_all[mask_model]
            rf.set_params(n_estimators=80)
            rf.fit(X_model, y_model)

    # 评估
    Xtr, Xte, ytr, yte = train_test_split(X_all, y_all, test_size=0.3, random_state=42)
    metrics = {
        "子集名称": subset_name,
        "特征数量": len(feature_subset),
        "R2_in": r2_score(ytr, rf.predict(Xtr)),
        "R2_out": r2_score(yte, rf.predict(Xte)),
        "EVS_out": explained_variance_score(yte, rf.predict(Xte)),
        "MSE_out": mean_squared_error(yte, rf.predict(Xte)),
        "MAE_out": mean_absolute_error(yte, rf.predict(Xte)),
        "MedAE_out": median_absolute_error(yte, rf.predict(Xte)),
    }
    return metrics

In [25]:
linear_feats = ['fet','pe']
nonlinear_feats = ['fee','penr']
shap_top5_feats = ["fee", "penr", "fet", "pe", "feg","pen","eced","pegwoc","fel","peg"]

In [26]:
target_var = 'see'
model_name = "IMACLIM 1.1"  # 👈 你传入的模型名
outdir = os.path.join(OUTPUT_DIR, "see")
os.makedirs(outdir, exist_ok=True)

results = []
results.append(evaluate_subset(target_var, df, linear_feats, "线性主效变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, nonlinear_feats, "非线性交互变量", outdir, model_name))
results.append(evaluate_subset(target_var, df, shap_top5_feats, "SHAP前十变量", outdir, model_name))

# 保存结果
summary_df = pd.DataFrame(results)
summary_df.to_csv(os.path.join(outdir, f"{model_name}_{target_var}_子集建模比较结果.csv"), index=False, encoding='utf-8-sig')
print("✅ 子集建模评估完成，结果已保存至：", outdir)

✅ 子集建模评估完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see


#### Sobol 漏斗式验证

In [27]:
import pandas as pd
df = pd.read_csv(r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\data\var_attri_data_interp_cleaned.csv")
# 假设你的变量名叫 'CO2'
X = df[['fee','penr']]        # 用你关心的变量
y = df['see']

output_dir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results"
var_names = ['fee','penr']

# 针对模型子样本
df_gcam = df[df['Model'] == 'IMACLIM 1.1']
X_gcam = df_gcam[var_names]
y_gcam = df_gcam['see']
run_sobol_analysis(X_gcam, y_gcam, var_names, output_dir, tag="see", target_model='IMACLIM 1.1')


C:\Users\phc\AppData\Local\Temp\ipykernel_17212\2532936671.py:18: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, 512, calc_second_order=True)


[✔] Sobol 总效应分析完成，结果已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see_model_IMACLIM 1.1_sobol.csv


C:\Users\phc\AppData\Roaming\Python\Python310\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


#### 模型降维&交互建模

##### fee

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ['fee']
target_var = 'see'
model_name = "IMACLIM 1.1"
outdir = os.path.join(OUTPUT_DIR, "see")
os.makedirs(outdir, exist_ok=True)

# 仅保留指定模型的数据
df_model = df[df['Model'] == model_name]
X = df_model[feature_subset].dropna()
y = df_model.loc[X.index, target_var]

# === 拆分训练/测试集 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 训练集上建模
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本增量微调
china_mask = df_model.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. IMACLIM 1.1 子样本增量微调
model_mask = (df_model.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 训练/测试集分指标评估
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}

summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF fee→see/IMACLIM 1.1 增量微调分训练/测试集回归评估，结果已保存至：", outdir)

✅ RF fee→see/IMACLIM 1.1 增量微调分训练/测试集回归评估，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see


##### fee+fee分段

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# === 构造 fee 分段 ===
df['fee_low']    = (df.fee <= 10).astype(int)
df['fee_medium'] = ((df.fee > 10) & (df.fee <= 25)).astype(int)
df['fee_high']   = (df.fee > 25).astype(int)

feature_subset = ['fee_low', 'fee_medium', 'fee_high', 'fee']
target_var = 'see'
model_name = "IMACLIM 1.1"
outdir = os.path.join(OUTPUT_DIR, "see")
os.makedirs(outdir, exist_ok=True)

# 仅保留指定模型的数据
df_model = df[df['Model'] == model_name]
X = df_model[feature_subset].dropna()
y = df_model.loc[X.index, target_var]

# === 拆分训练/测试集 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# 1. 训练集上建模
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# 2. 中国子样本增量微调
china_mask = df_model.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# 3. IMACLIM 1.1 子样本增量微调
model_mask = (df_model.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# 4. 训练/测试集分指标评估
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF fee分段+fee→see/IMACLIM 1.1 增量微调模型分训练/测试集评估已保存至：", outdir)

✅ RF fee分段+fee→see/IMACLIM 1.1 增量微调模型分训练/测试集评估已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see


##### fee+penr

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

feature_subset = ['fee', 'penr']
target_var = 'see'
model_name = "IMACLIM 1.1"
outdir = os.path.join(OUTPUT_DIR, "see")
os.makedirs(outdir, exist_ok=True)

# 仅保留指定模型的数据
df_model = df[df['Model'] == model_name]
X = df_model[feature_subset].dropna()
y = df_model.loc[X.index, target_var]

# === 拆分训练/测试集 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 全训练集建模 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本增量微调 ===
china_mask = df_model.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. IMACLIM 1.1 子样本增量微调 ===
model_mask = (df_model.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练/测试集评估六项指标 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF fee+penr→see/IMACLIM 1.1 增量微调模型分训练/测试集评估已保存至：", outdir)

✅ RF fee+penr→see/IMACLIM 1.1 增量微调模型分训练/测试集评估已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see


##### fee+penr+penr*peowoc

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 构造交互项
df['penr_peowoc'] = df['penr'] * df['peowoc']

feature_subset = ['fee', 'penr', 'penr_peowoc']
target_var = 'see'
model_name = "IMACLIM 1.1"
outdir = os.path.join(OUTPUT_DIR, "see")
os.makedirs(outdir, exist_ok=True)

# 仅保留指定模型的数据
df_model = df[df['Model'] == model_name]
X = df_model[feature_subset].dropna()
y = df_model.loc[X.index, target_var]

# === 训练/测试集拆分 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 全训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本增量微调 ===
china_mask = df_model.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. IMACLIM 1.1 子样本增量微调 ===
model_mask = (df_model.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练集、测试集评估6项指标 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调回归结果.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF fee+penr+penr*peowoc→see/IMACLIM 1.1 增量微调，分训练/测试集六指标已保存至：", outdir)

✅ RF fee+penr+penr*peowoc→see/IMACLIM 1.1 增量微调，分训练/测试集六指标已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see


##### fee+penr+fee*peowoc

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 构造交互项
df['fee_peowoc'] = df['fee'] * df['peowoc']

feature_subset = ['fee', 'penr', 'fee_peowoc']
target_var = 'see'
model_name = "IMACLIM 1.1"
outdir = os.path.join(OUTPUT_DIR, "see")
os.makedirs(outdir, exist_ok=True)

# 仅保留指定模型的数据
df_model = df[df['Model'] == model_name]
X = df_model[feature_subset].dropna()
y = df_model.loc[X.index, target_var]

# === 训练/测试集拆分 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 全训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本增量微调 ===
china_mask = df_model.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. IMACLIM 1.1 子样本增量微调 ===
model_mask = (df_model.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练/测试集六项指标输出 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调回归六指标.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF fee+penr+fee*peowoc→see/IMACLIM 1.1 六项指标输出已完成，结果已保存至：", outdir)

✅ RF fee+penr+fee*peowoc→see/IMACLIM 1.1 六项指标输出已完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see


##### fee+penr+penr*peowoc+fee*peowoc

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    r2_score, explained_variance_score, mean_squared_error,
    mean_absolute_error, median_absolute_error
)
from sklearn.model_selection import train_test_split
import pandas as pd
import os

def calc_metrics(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "EVS": explained_variance_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MedAE": median_absolute_error(y_true, y_pred),
    }

# === 加载数据 ===
df = pd.read_csv(INPUT, encoding='utf-8-sig')
df['Year'] = df['Year'].astype(int)

# 构造交互项
df['penr_peowoc'] = df['penr'] * df['peowoc']
df['fee_peowoc']  = df['fee']  * df['peowoc']

feature_subset = ['fee', 'penr', 'penr_peowoc', 'fee_peowoc']
target_var = 'see'
model_name = "IMACLIM 1.1"
outdir = os.path.join(OUTPUT_DIR, "see")
os.makedirs(outdir, exist_ok=True)

# 仅分析指定模型
df_model = df[df['Model'] == model_name]
X = df_model[feature_subset].dropna()
y = df_model.loc[X.index, target_var]

# === 拆分训练集和测试集 ===
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)

# === 1. 全训练集训练 ===
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    min_samples_leaf=4,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1,
    warm_start=True
)
rf.fit(Xtr, ytr)

# === 2. 中国子样本增量微调 ===
china_mask = df_model.loc[Xtr.index, 'Region'].isin(["CHN", "R10CHINA+"])
if china_mask.sum() > 30:
    X_china = Xtr[china_mask]
    y_china = ytr[china_mask]
    rf.set_params(n_estimators=150)
    rf.fit(X_china, y_china)

# === 3. IMACLIM 1.1 子样本增量微调 ===
model_mask = (df_model.loc[Xtr.index, 'Model'] == model_name)
if model_mask.sum() > 30:
    X_model = Xtr[model_mask]
    y_model = ytr[model_mask]
    rf.set_params(n_estimators=200)
    rf.fit(X_model, y_model)

# === 4. 训练/测试集六项指标输出 ===
metrics_in  = calc_metrics(ytr, rf.predict(Xtr))
metrics_out = calc_metrics(yte, rf.predict(Xte))

summary = {
    "样本": "全部",
    "R2_in":     metrics_in["R2"],
    "R2_out":    metrics_out["R2"],
    "EVS_out":   metrics_out["EVS"],
    "MSE_out":   metrics_out["MSE"],
    "MAE_out":   metrics_out["MAE"],
    "MedAE_out": metrics_out["MedAE"],
}
summary_df = pd.DataFrame([summary])
summary_df.to_csv(
    os.path.join(outdir, f"{target_var}_{'+'.join(feature_subset)}_RF增量微调回归六指标.csv"),
    index=False,
    encoding='utf-8-sig'
)
print("✅ RF fee+penr+penr*peowoc+fee*peowoc→see/IMACLIM 1.1 六项指标输出已完成，结果已保存至：", outdir)

✅ RF fee+penr+penr*peowoc+fee*peowoc→see/IMACLIM 1.1 六项指标输出已完成，结果已保存至： C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\RF\model_attri\results\see
